<a href="https://colab.research.google.com/github/Renator12/Structured-products/blob/main/structuredprods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pricing With A Willow Tree

In [92]:
from statistics import NormalDist
from scipy.integrate import quad
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.optimize import NonlinearConstraint
from scipy.stats import norm
from scipy.optimize import minimize_scalar
import random


In [93]:
# Global Variables
S0 = 100 #Initial Stock Price
K = 103
r = 0.05 # interest rate
sigma = 0.40 # Volatility
years = 6
time_periods = 1# per year
branch_count = 6# Only use even numbers
dt = 1/time_periods

### Discrete Approximations $z_i$

#### Xu et. al Method

##### Generate Q Values

In [94]:
q_value = []


phi = 0.6 # A value from 0 to 1

if branch_count == 2:
    q_value.append((1-0.5)**phi/branch_count)
elif branch_count % 2 == 0:
    for i in range(1,int(branch_count/2)+1):
        q_value.append((i-0.5)**phi/branch_count)

elif branch_count % 2 != 0:
    for i in range(1,int((branch_count+1)/2)):
        q_value.append((i-0.5)**phi/branch_count)

q_sum = sum(q_value)*2
q_transform = [x / q_sum for x in q_value]
q_full = q_transform + q_transform[::-1]
# q vector (coefficients for the zi terms)
qi = np.array(q_full)

##### Defining Optimization Parameters

In [95]:
big_z = [-np.inf]

for i in range(1,len(q_transform)+1):
    big_z.append(NormalDist().inv_cdf(sum(q_transform[0:i])))

big_z = big_z + [x * -1 for x in big_z[len(big_z)-2::-1]]



# Zi vector (bounds for the zi variables)
Z_lower = np.array(big_z[0:-1])
Z_upper = np.array(big_z[1:])



# The objective function we want to minimize
def objective_function(z):
    return (np.sum(qi * z**4) - 3)**2

# The equality constraints
def constraint1(z):
    return np.sum(qi * z)

def constraint2(z):
    return np.sum(qi * z**2) - 1

# Define the constraints dictionary
con1 = {'type': 'eq', 'fun': constraint1}
con2 = {'type': 'eq', 'fun': constraint2}
constraints = [con1, con2]


# Define the bounds for each zi
bounds = [((None,high) if low == -np.inf else ((low,None) if high == np.inf else (low, high)) ) for low, high in zip(Z_lower, Z_upper)]

# # Define the bounds for each zi
# bounds = [((None,high) if low == -np.inf else (low, high)) for low, high in zip(Z_lower, Z_upper)]

# Initial guess (required by the minimizer, let's start with the midpoint of the bounds)
z_initial = (Z_lower + Z_upper) / 2
z_initial[0] = NormalDist().inv_cdf(sum(q_full[0:1])/2)
z_initial[-1] = z_initial[0]*-1

##### SLSQP Optimization Method

In [96]:
# Run the minimizer
result = minimize(objective_function, z_initial,method='SLSQP', bounds=bounds, constraints=constraints)

# The result of minimization will be in result.x
result.x, result.success, result.message

z_value = result.x

### Brownian Motion Tree

In [97]:
def generate_btree(time_periods,years,branch_count,z_value):
    b_tree = np.zeros((time_periods*years+1,branch_count))

    for i in range (1,time_periods*years+1):
        for j in range (0,branch_count):
            b_tree[i][j] = np.sqrt(i*dt) * z_value[j]
    return b_tree

### Conditional Expectation Function

#### w1 = $\tilde{W}_{j+1}^{n+1}$ w2 = $\tilde{W}_{j}^{n+1}$ w3 = $\tilde{W}_{j-1}^{n+1}$

In [98]:
def probcali(p, y, mu, sigma):

    m = len(p)
    a = np.dot(p, y) - mu
    b = np.dot(p, y**2) - mu**2 - sigma**2
    pind = np.argsort(p)  # Sorts p and returns the indices that would sort p

    x = np.zeros(m)
    wm = pind[-1]  # Index of max probability
    w2 = pind[-2]
    w1 = pind[-3]

    y1 = y[w1]  # Min
    y2 = y[w2]  # Middle
    ym = y[wm]  # Max probability

    # Calculate calibration factors
    x[wm] = (-b + a * (y1 + y2)) / (ym**2 - y2**2 - (y1 + y2) * (ym - y2))
    x[w1] = (-a - x[wm] * (ym - y2)) / (y1 - y2)
    x[w2] = -(x[wm] + x[w1])

    pc = p + x  # Adjust original probabilities
    pc = np.maximum(pc, 0)  # Ensure no negative probabilities
    pc /= np.sum(pc)  # Normalize to ensure the sum of probabilities is 1

    return pc

In [99]:
def generate_transition_matrices(time_periods,years,branch_count,b_tree):
    transition_matrices = []

    # n-1 number of matrices
    for n in range(1, time_periods*years):
        # intialize new matrix
        transition_matrix = np.zeros((branch_count, branch_count))
        test = np.zeros((branch_count, branch_count))
        # ith columns and jth rows
        for i in range(0, branch_count):
            # j goes from 0 to number of branches
            for j in range(0, branch_count):
                if j == 0:
                    w3 = np.NINF
                else:
                    w3 = b_tree[n+1][j-1]
                if j == (branch_count-1):
                    w1 = np.inf
                else:
                    w1 = b_tree[n+1][j+1]

                w2 = b_tree[n+1][j]

                upper_bound = (w1 + w2)/2
                lower_bound = (w2 + w3)/2

                w = b_tree[n][i]

                # Check built in cdf
                f = lambda x: 1/np.sqrt(2*np.pi*dt)*np.exp(-(x-w)**2/(2*dt))

                # Use the standard normal pdf to integrate from lower to upper bound
                transition_matrix[i][j], _ = quad(f, lower_bound, upper_bound)
        for i in range(branch_count):

            mu = b_tree[n][i]
            sigma = np.sqrt(dt)
            y = b_tree[n+1]  # Assuming y values for calibration are the next step's node values

            # Calibrate probabilities
            transition_matrix[i, :] = probcali(transition_matrix[i, :], y, mu, sigma)
            # Ensure probabilities sum to 1 after calibration
            transition_matrix[i, :] /= np.sum(transition_matrix[i, :])

        transition_matrices.append(transition_matrix)

        # Append the calculated matrix to the list of transition matrices
        transition_matrices.append(transition_matrix)
    return transition_matrices

### Stock Price Tree

In [100]:
def generate_stock_price_tree(S0, r, sigma, dt, branch_count, time_periods,years, z_value):

    stock_price_tree = np.zeros((time_periods*years+1, branch_count))

    # Populate the stock price tree
    for i in range(1, time_periods*years+1):
        for j in range(branch_count):
            # Transform the Brownian motion value into a stock price
            W_t = np.sqrt(i * dt) * z_value[j]  # Brownian motion at time t
            stock_price_tree[i][j] = S0 * np.exp((r - 0.5 * sigma**2) * i * dt + sigma * W_t)

    # Set the initial stock prices
    stock_price_tree[0] = S0

    return stock_price_tree

### Payoff Calculations

#### Option Pricing

In [101]:
def price_option(stock_price_tree, transition_matrices, K, r, dt, is_call=True):


    # Initialize option value tree with zeros
    option_value_tree = np.zeros_like(stock_price_tree)

    # Determine the option value at the final time step
    if is_call:
        option_value_tree[-1] = np.maximum(stock_price_tree[-1] - K, 0)
    else:
        option_value_tree[-1] = np.maximum(K - stock_price_tree[-1], 0)

    # Backward induction to calculate option price at each node
    for t in range(time_periods*years-1, 0, -1):
        option_value_tree[t] = np.dot(transition_matrices[t-1],option_value_tree[t+1])*np.exp(-r*dt)

    option_value_tree[0] = np.dot(qi,option_value_tree[1])

    # The option price is the value at the initial node
    return option_value_tree[0][0]

In [102]:
def BS_CALL(S, K, T, r, sigma):
    N = norm.cdf
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * N(d1) - K * np.exp(-r*T)* N(d2)

In [103]:
b_tree = generate_btree(time_periods,years,branch_count,z_value)
transition_matrices = generate_transition_matrices(time_periods,years,branch_count,b_tree)

In [104]:
np.array(transition_matrices).shape

(10, 6, 6)

In [105]:
transition_matrices

[array([[5.71463380e-01, 2.94797861e-01, 1.10566394e-01, 2.06494131e-02,
         2.48916253e-03, 3.37896980e-05],
        [1.21614082e-01, 3.97191627e-01, 3.05508814e-01, 1.24902032e-01,
         4.83662516e-02, 2.41719357e-03],
        [4.74853949e-02, 2.71078616e-01, 3.19522495e-01, 2.33625284e-01,
         1.17984002e-01, 1.03042078e-02],
        [1.03041874e-02, 1.17984025e-01, 2.33625362e-01, 3.19522448e-01,
         2.71078509e-01, 4.74854686e-02],
        [2.41718886e-03, 4.83662701e-02, 1.24902181e-01, 3.05508666e-01,
         3.97191486e-01, 1.21614208e-01],
        [3.37897445e-05, 2.48917097e-03, 2.06494687e-02, 1.10566971e-01,
         2.94797056e-01, 5.71463543e-01]]),
 array([[5.71463380e-01, 2.94797861e-01, 1.10566394e-01, 2.06494131e-02,
         2.48916253e-03, 3.37896980e-05],
        [1.21614082e-01, 3.97191627e-01, 3.05508814e-01, 1.24902032e-01,
         4.83662516e-02, 2.41719357e-03],
        [4.74853949e-02, 2.71078616e-01, 3.19522495e-01, 2.33625284e-01,
     

In [106]:
prices = generate_stock_price_tree(S0, r, sigma, dt, branch_count, time_periods,years, z_value)

In [107]:
prices

array([[100.        , 100.        , 100.        , 100.        ,
        100.        , 100.        ],
       [ 43.74049138,  69.75491791,  85.29378601, 110.41421256,
        135.01049175, 215.30718753],
       [ 30.51410832,  59.04040003,  78.46397841, 113.03537906,
        150.22264315, 290.6589985 ],
       [ 22.98647352,  51.5874144 ,  73.08453924, 114.28824149,
        161.91357673, 363.37445401],
       [ 18.01812711,  45.82389436,  68.5136517 , 114.8133239 ,
        171.66326533, 436.57552712],
       [ 14.48707251,  41.13503566,  64.49347533, 114.86718184,
        180.09423963, 511.36464268],
       [ 11.86012298,  37.20310132,  60.88689826, 114.58565079,
        187.53178631, 588.25338869]])

In [108]:
np.array(prices).shape

(7, 6)

In [109]:
# Global Variables

option_type = 'call'
exercise_style = 'European'
price_option(prices, transition_matrices, K, r, dt, is_call=True)

47.62679528936913

In [110]:
BS_CALL(S0,K,years,r,sigma)

46.122966171746455

#### Structured Product Pricing

##### Updated Method

In [111]:
def payoff_selection(function,fixed_rate=0,returns=0,excess_ratio=0,participation_rate=1):
    if function == 'auto_call':
        return (1+fixed_rate+excess_ratio*(returns-fixed_rate))
    elif function == 'boost':
        return (1+max(fixed_rate,fixed_rate+(returns-fixed_rate)*participation_rate))

def payoffs(prices,fixed_rate,payoff_function,participation_rate=1,boost_level=0,excess_ratio=0,principal_protected=True,barrier_rate=1,geared_protection=False,gearing_rate=1,gearing_fixed=0,final=False):

    returns = (prices-S0)/S0

    count = 0
    for i in returns:

        if i <= 0:
            # Boosted return if returns are negative but above boost_level
            if i >= boost_level:
                returns[count] = (S0 * payoff_selection(payoff_function,fixed_rate))
            # No return if the rate of return is negative and pricipal protected or above barrier rate
            elif (principal_protected or (i >= (barrier_rate-1))):
                returns[count] = S0
            # Geared protection adds a fixed rate to the negative return and multiplies by gearing ratio
            elif geared_protection:
                returns[count] = S0*(1+(i+gearing_fixed)*gearing_rate)
            # If its the final payoff and all else fails just return the payoffs
            elif final:
                returns[count] = S0*(1+i)
            # Return 0 otherwise
            else:
                returns[count] = 0
        elif i < fixed_rate:
            returns[count] = (S0 * payoff_selection(payoff_function,fixed_rate))  # Fixed rate if the return is less than fixed rate
        else:
            # Apply the structured product logic if return is above the fixed rate
            returns[count] =  (S0 * payoff_selection(payoff_function,fixed_rate,i,excess_ratio,participation_rate))
        count += 1
    return returns

# **CUSTOM PAYOFFS**

In [112]:
def payoffs2(prices,S0 , fixed_rate,excess_return_variable):#autocall 105% ->input 1.05
  returns = (prices-S0)/S0
  count=0
  for i in returns:
    if i>0:
      excess_ratio=max(0,(i-fixed_rate)*excess_return_variable)
      #print(excess_ratio)
      returns[count]=S0*(1+fixed_rate+excess_ratio)
    else:
      returns[count]=S0
    count+=1
  return returns

In [113]:
def price_auto_call2(stock_price_tree, transition_matrices, r, fixed, variable, years,excess_ratio_variable,auto_call ):
    '''
    price_auto_call2 : Uses a willow tree method to price a structured product with varying fixed rates per year , increasing at a constant rate and having an autocall value.Payoff is given either on autocall day where
    Payoff=S0*(1+Fixed_rate + excess_ratio) .
    '''
    year_count = years - 1
    Final_rate = fixed + (year_count) * variable  # Calculate the initial Final Rate
    # Initialize option value tree
    option_value_tree = np.zeros_like(stock_price_tree)
    prices=stock_price_tree[-1]
    # Calculate payoffs at maturity (final period)
    option_value_tree[-1] = payoffs2(prices,S0 , fixed,excess_ratio_variable)
    # Backward induction for each time step
    for t in range(time_periods * years - 1, 0, -1):
        payoff = payoffs2(stock_price_tree[t],S0,Final_rate,excess_ratio_variable)
        dot_prod = np.dot(transition_matrices[t - 1], option_value_tree[t + 1]) * np.exp(-r * dt)
        option_value_tree[t] = np.where(payoff >S0*auto_call, payoff,dot_prod)
        # Reduce the Final_rate at every auto-callable date (each period)
        #print(Final_rate)
        Final_rate -= variable  # Decrease Final_rate by the specified decrement

    option_value_tree[0] = np.dot(qi, option_value_tree[1]) * np.exp(-r * dt)
   #print('final rate is ',Final_rate)
    return option_value_tree[0][0]

In [114]:
np.array(transition_matrices[0]).shape

(6, 6)

In [115]:
np.array(prices).shape

(7, 6)

In [116]:
prices=generate_stock_price_tree(S0, r, 0.2, dt, branch_count, time_periods,years, z_value)
auto_call_price = price_auto_call2(prices,transition_matrices,r=0.03,fixed=0.05,excess_ratio_variable=0.05,years=years,variable=0.05,auto_call=1.05)
auto_call_price

104.86982504183068

In [117]:
prices.shape

(7, 6)

In [118]:
def price_asian_prod(stock_price_tree, transition_matrices, r, S0, years,time_periods,participation_rate=1):

    '''
    price_asian_prod : Uses a willow tree method to price an structured product with payoffs based on the final average index and its return.Pricing done similarly to Asian options
    using our interpolation sceheme as highlighted and covered in the paper 'An efficient convergent lattice method for Asian option pricing with superlinear complexity'

    parameters :
    stock_price_tree : prices per time period created using the function 'generate_stock_price_tree' using GBM.
    transition_matrices : transitional probability matrices creating using 'generate_transition_matrices' .
    r : risk-free rate
    S0: initial index/stock value
    years :number of years
    time_periods :periods per year
    pariticipation_rate : percentage of the performance of the underlying asset which will be used to calculate the return when a product reaches its maturity.
    '''
    m=len(stock_price_tree[0]) #number of nodes per time period
    year_count = years - 1
    option_value_tree = np.zeros_like(stock_price_tree)
    k_min,k_max,avg_k_final=get_return(time_periods*years-1,stock_price_tree)
    var_return=find_return_from_alpha(S0,avg_k_final,participation_rate)
    option_value_tree[-1]=var_return #final layer
    for n in range(time_periods * years - 1, 0, -1):
       for k in range(k_min,k_max+1):
        A_k_n=get_A_k(k,S0)
        for i in range(1,m+1):
          V_n_ik=0
          for j in range(1,m+1):
            A_tilda_jkn=A_k_n+(stock_price_tree[n+1][j]-A_k_n)/(n+2)
            A_k_nplusone=get_A_k(k,S0)
            avg_min,avg_max,kmin_new,kmax_new=find_alpha(S0,stock_price_tree,A_k_nplusone,k_min,k_max,n) #finding A_K- AND A_K+
            alpha_jk_n=(A_tilda_jkn-avg_min)/(avg_max-avg_min)
            V_kminus=find__return_from_alpha(S0,get_A_k(k_min_new,S0),participation_rate)
            V_kplus=find_return_from_alpha(S0,get_A_k(k_max_new,S0),participation_rate)
            V_jk_n=alpha_jk_n*V_kplus+(1-alpha_jk_n)*V_kminus
            V_n_ik+=transition_matrices[n-1][i-1][j-1]*alpha_jk_n*V_jk_n
          option_value_tree[n][i]=np.exp(-r*dt)*V_n_ik
    #for j in range(1,m+1):
      #A_tilda_jkn_1=(S0+stock_price_tree[1][j])/2
      #V_jk_one=
def get_A_k(k,S0):
  A_k=S0*np.exp(k)
  return A_k
def get_return(n,stock_prices):
  A_min_n = (1 / (n + 1)) * (S0 + np.sum(np.min(stock_prices, axis=1)))
  A_max_n = (1 / (n + 1)) * (S0 + np.sum(np.max(stock_prices, axis=1)))
  k_min_n = np.floor(np.log(A_min_n / S0))
  k_max_n = np.ceil(np.log(A_max_n / S0))
  random_k=random.randint(k_min_n,k_max_n)
  A_k_n=get_A_k(random_k,S0)
  #avg_return=(A_k_n-S0)/S0
  #our_return=avg_return*participation_rate*S0
  return (k_min_n,k_max_n,A_k_n)

  #return avg_return*participation_rate*S0 if our_return>0 else S0
def find_alpha(S0,stock_prices,A_jk , k_min , k_max , n):
  kmin_temp=k_min
  kmax_temp=k_max
  A_k_min=get_A_k(kmin_temp,S0)
  A_k_max=get_A_k(kmax_temp,S0)
  while(A_k_min<=A_jk and kmin_temp<k_max):
    A_k_min_potential=get_A_k(kmin_temp,S0)
    if A_k_min_potential>A_jk:
      break
    A_k_min=A_k_min_potential
    kmin_temp+=1
  while(A_k_max>=A_jk and kmax_temp>k_min):
    A_k_max_potential=get_A_k(kmax_temp,S0)
    if A_k_max_potential<A_jk:
      break
    A_k_max=A_k_max_potential
    kmax_temp-=1
  return (A_k_min,A_k_max,kmin_temp,kmax_temp)

def find_return_from_alpha(S0,avg,participation_rate=1):
  index_return=(avg-S0)/S0
  return index_return*participation_rate*S0



In [119]:
#get_return(years*time_periods,prices[:4])
a=find_alpha(100,prices,99,-5,8,3)
print(a[1])
find_return_from_alpha(S0,a[0],0.8)

100.0


-50.56964470628461

In [120]:
#my custom payoffs function 2

def price_booster_BASIC_PPN(stock_price_tree,transition_matrices,r):
    moving_sum=0
    # Initialize option value tree with zeros
    option_value_tree = np.zeros_like(stock_price_tree)

    # Payoff function takes in the stock prices and a fixed rate to get a payoff vector
    option_value_tree[-1] = (stock_price_tree[-1]-S0)/S0
    print(f'Option value trhee at -1 is {option_value_tree[-1]}')
    # Backward induction to calculate option price at each node
    for t in range(time_periods*years-1, 0, -1):
        option_value_tree[t] = np.dot(transition_matrices[t-1],option_value_tree[t+1])*np.exp(-r*dt)
        print(transition_matrices[t-1])
        #print(f'Option value tree at {t} is {option_value_tree[t]}')

        moving_sum+=option_value_tree[t]
    option_value_tree[0] = np.dot(qi,option_value_tree[1])*np.exp(-r*dt)
    #moving_sum+=(option_value_tree[t]-S0)/S0
    print(option_value_tree[0][0])
    return option_value_tree[0][0]
#price_booster_BASIC_PPN(prices,transition_matrices,r)


In [121]:
transition_matrices[-1].shape

(6, 6)

In [122]:
def price_auto_call(stock_price_tree,transition_matrices,r,fixed,variable,years,excess_ratio=0,principal_protected=False,barrier_rate=1,geared_protection=False,gearing_rate=1,gearing_fixed=0):
    year_count = years-1

    Final_rate = fixed + year_count*variable

    # Initialize option value tree with zeros
    option_value_tree = np.zeros_like(stock_price_tree)

    # Payoff function takes in the stock prices and a fixed rate to get a payoff vector
    option_value_tree[-1] = payoffs(stock_price_tree[-1],Final_rate,'auto_call',excess_ratio,principal_protected,barrier_rate,geared_protection,gearing_rate,gearing_fixed,True)

    # Backward induction to calculate option price at each node
    for t in range(time_periods*years-1, 0, -1):
        print('option_value_tree at ',t+1,':',option_value_tree[t+1].shape)
        print('transition_matrices at ',t,':',transition_matrices[t-1].shape)
        option_value_tree[t] = np.dot(transition_matrices[t-1],option_value_tree[t+1])*np.exp(-r*dt)

        if t == time_periods-1: # when to do american call payoff
            payoff = payoffs(stock_price_tree[t],Final_rate,'auto_call',excess_ratio,principal_protected,barrier_rate,geared_protection,gearing_rate,gearing_fixed)
            option_value_tree[t] = np.where(payoff == 0,option_value_tree[t],payoff)

    option_value_tree[0] = np.dot(qi,option_value_tree[1])*np.exp(-r*dt)

    return option_value_tree[0][0]

In [123]:
def price_booster(stock_price_tree,transition_matrices,r,fixed_portion,years,participation_rate=1,boost_level=0,excess_ratio=0,principal_protected=False,barrier_rate=1,geared_protection=False,gearing_rate=1,gearing_fixed=0):

    # Initialize option value tree with zeros
    option_value_tree = np.zeros_like(stock_price_tree)

    # Payoff function takes in the stock prices and a fixed rate to get a payoff vector
    option_value_tree[-1] = payoffs(stock_price_tree[-1],fixed_portion,'boost',participation_rate,boost_level,excess_ratio,principal_protected,barrier_rate,geared_protection,gearing_rate,gearing_fixed,True)

    # Backward induction to calculate option price at each node
    for t in range(time_periods*years-1, 0, -1):
        option_value_tree[t] = np.dot(transition_matrices[t-1],option_value_tree[t+1])*np.exp(-r*dt)

    option_value_tree[0] = np.dot(qi,option_value_tree[1])*np.exp(-r*dt)

    return option_value_tree[0][0]

In [124]:
r=0.05

In [136]:
a=np.array([1,2,3,4]).transpose()
b=2
np.dot(a,b)

array([2, 4, 6, 8])

In [125]:

b_tree = generate_btree(time_periods,years,branch_count,z_value)
transition_matrices = generate_transition_matrices(time_periods,years,branch_count,b_tree)

In [126]:
auto_callprice=price_auto_call(prices,transition_matrices,r,0.5,0.05,years,excess_ratio=0,principal_protected=False,barrier_rate=1,geared_protection=False,gearing_rate=1,gearing_fixed=0)

option_value_tree at  6 : (6,)
transition_matrices at  5 : (6, 6)
option_value_tree at  5 : (6,)
transition_matrices at  4 : (6, 6)
option_value_tree at  4 : (6,)
transition_matrices at  3 : (6, 6)
option_value_tree at  3 : (6,)
transition_matrices at  2 : (6, 6)
option_value_tree at  2 : (6,)
transition_matrices at  1 : (6, 6)


In [127]:
prices=generate_stock_price_tree(S0, r, sigma, dt, branch_count, 2,5, z_value)
booster_price = price_booster(prices,transition_matrices,0.03,0.14,5,boost_level=0,barrier_rate=0.8)
booster_price

0.0

In [128]:
prices = generate_stock_price_tree(S0, r, sigma, dt, branch_count, time_periods,5, z_value)
auto_call_price = price_auto_call2(prices,transition_matrices,r=0.03,fixed=0.05,excess_ratio_variable=0.05,years=5,variable=0.05,auto_call=1.05)
auto_call_price

103.01629670361824

In [129]:
prices = generate_stock_price_tree(S0, r, sigma, dt, branch_count, time_periods,5, z_value)
model_price = price_booster(prices,transition_matrices,0.03,0.7,5,boost_level=-.10,barrier_rate=0.8)
print(model_price)

120.4187252566285


### Implied Volatility

In [130]:
from scipy.optimize import brentq,newton

# Define a function to find the implied volatility
def find_implied_volatility(sigma,market_price,years):

    def difference(sigma):
        prices = generate_stock_price_tree(S0, r, sigma, dt, branch_count, time_periods,years, z_value)

        # Parameters for auto call function
        # price_auto_call(stock_price_tree,transition_matrices,r,fixed,variable,years,excess_ratio=0,principal_protected=False,barrier_rate=1,geared_protection=False,gearing_rate=1,gearing_fixed=0):

        # model_price = price_auto_call(prices,transition_matrices,r=0.03,fixed=0.1,variable=0.05,years)
        model_price = price_auto_call2(prices, transition_matrices, r, 0.05, 0.05, 6,0.05,1.0)
        #print(model_price)
        difference = (model_price - market_price)
        print('model price is ',model_price , 'market price ius ',market_price)
        return difference

    # Use the Brent method to find the root of the difference function
    sigma = brentq(difference, a=sigma, b=2, xtol=1e-6)


    return sigma

In [131]:
from scipy.optimize import brentq

# Define a function to find the implied volatility
def find_implied_volatility_og(sigma,market_price,years):

    def difference(sigma):
        prices = generate_stock_price_tree(S0, r, sigma, dt, branch_count, time_periods,years, z_value)

        # Parameters for auto call function
        # price_auto_call(stock_price_tree,transition_matrices,r,fixed,variable,years,excess_ratio=0,principal_protected=False,barrier_rate=1,geared_protection=False,gearing_rate=1,gearing_fixed=0):

        # model_price = price_auto_call(prices,transition_matrices,r=0.03,fixed=0.1,variable=0.05,years)
        model_price = price_booster(prices,transition_matrices,0.03,0.7,5,boost_level=-.10,barrier_rate=0.8)

        difference = (model_price - market_price)
        print('model price is ',model_price , 'market price ius ',market_price)
        return difference

    # Use the Brent method to find the root of the difference function
    sigma = brentq(difference, a=sigma, b=2, xtol=1e-6)
    return sigma

In [132]:
# Given market price of the option
market_price = 101.25 # Insert the market price here
# Find the implied volatility
implied_volatility = find_implied_volatility(sigma,market_price,years) #sample example https://notes.tdsecurities.com/api/v1/notes/file/3618/information_statement/EN
implied_volatility

model price is  98.35248296946091 market price ius  101.25
model price is  83.20896571059666 market price ius  101.25


ValueError: f(a) and f(b) must have different signs

In [ ]:
# Given market price of the option
# Find the implied volatility
implied_volatility = find_implied_volatility_og(sigma,market_price,5)
implied_volatility